In [2]:
###################################################################################
# CURRENT NOISE MODEL FOR RETURNING DT OF ALL SPECTRA
###################################################################################

from astropy.io import ascii 
from astropy.table import Table
from astropy.table import Column
import numpy as np 
import coronagraph as cg
import matplotlib.pyplot as plt
import math
import os, glob
import copy
import collections
import pandas as pd

os.chdir('/Users/schuylerraeschroerborges/Desktop/Second_Project/')

def newcols(data):
    columnone=[]
    columntwo=[]
    columnfour=[]
    columnthree=[]
    if data.endswith('.rad'):
        clear = ascii.read(data, data_start=0,  delimiter='\s')
        for i in range (0,len(clear)):
            columnone.append(float(clear[i][0]))
            columnfour.append(float(clear[i][3]))
            columnthree.append(float(clear[i][2]))
        wave=np.array(columnone)
        rad=np.array(columnfour)
        flux=np.array(columnthree)
    
        reflect=np.divide(rad,flux)        
    else:
        notclear = ascii.read(data, data_start=0,  delimiter='\s')
        for i in range (0,len(notclear)):
            columnone.append(float(notclear[i][0]))
            columntwo.append(float(notclear[i][1]))
            columnthree.append(float(notclear[i][2]))
            columnfour.append(float(notclear[i][3]))
        wave=np.array(columnone)
        flux=np.array(columntwo)
        reflect=np.array(columnthree)
        rad=np.array(columnfour)

        

    
    
    return np.array([wave,flux,rad,reflect])



In [4]:

file_list = (['Arthro_clearSky_mon_toa.rad', 'Arthro_clearSky_scrub_v3_um_toa.rad', \
              'Arthro_mixed25.txt', 'Arthro_new_mixed25_scrubbed.txt', 'Arthro_mixed50.txt', \
              'Arthro_new_mixed50_scrubbed.txt', 'Arthro_mixed75.txt', 'Arthro_new_mixed75_scrubbed.txt',\
              'Ecto_clearSky_new_toa.rad', 'Ectoth_clearSky_scrubbed_mon_toa.rad', 'Ectoth_mixed25.txt',\
              'Ectoth_mixed25_scrubbed.txt', 'Ectoth_mixed50.txt', 'Ectoth_mixed50_scrubbed.txt',\
              'Ectoth_mixed75.txt', 'Ectoth_mixed75_scrubbed.txt', 'black_clearSky_mon_toa.rad',\
              'black_clearSky_scrubbed_mon_toa.rad', 'blackmat_mixed25.txt', 'blackmat_mixed25_scrubbed.txt',\
              'blackmat_mixed50.txt', 'blackmat_mixed50_scrubbed.txt', 'blackmat_mixed75.txt',\
              'blackmat_mixed75_scrubbed.txt', 'orangemat_clearSky_mon_toa.rad',\
              'orangemat_clearSky_scrubbed_mon_toa.rad', 'orangemat_mixed25.txt', 'orangemat_mixed25_scrubbed.txt',\
              'orangemat_mixed50.txt', 'orangemat_mixed50_scrubbed.txt', 'orangemat_mixed75.txt',\
              'orangemat_mixed75_scrubbed.txt'])


outfile = "DetectionTimes_new_arthro_5pc.txt"
f = open(outfile, 'w')

for i in np.arange(0, len(file_list), 2):
    ax_file = file_list[i]
    ay_file = file_list[i+1]
    AX_data = newcols(ax_file)
    AY_data = newcols(ay_file)

    lamhr = np.array(AX_data[0])    # Lambda
    AX_solhr = np.array(AX_data[1])    # Solar flux AX 
    AX_radhr = np.array(AX_data[2])    # radiance AX
    AX_refhr = np.array(AX_data[3])    # Reflectance of AX

    AY_solhr = np.array(AY_data[1])    # Solar flux AY
    AY_radhr = np.array(AY_data[2])    # radiance AY
    AY_refhr = np.array(AY_data[3])    # Reflectance of AY
                        
                        
###################################################################################
# RETRIEVING CP0 FOR ALBEDO OF ONE
###################################################################################

    planet = cg.Planet()
    
    # Planet params
    planet.alpha = 90.     # phase angle at quadrature
    planet.Phi   = cg.teleplanstar.lambertPhaseFunction(1)      # phase function at quadrature (already included in SMART run)
    planet.Rp    = 1.0     # Earth radii
    planet.a     = 1.0     # semi-major axis (AU)

    # Stellar params
    star = cg.Star()
    star.Teff  = 5780.   # Sun-like Teff (K)
    star.Rs    = 1.      # star radius in solar radii
    

    # Planetary system params
    planet.distance = 5.    # distance to system (pc)
    planet.Nez  = 1.      # number of exo-zodis

    telescope = cg.Telescope()

    # Set paramaters for luvex
    telescope.diameter = 6.0
    telescope.contrast = 1e-10
    telescope.A_collect = 36.
    telescope.diam_inscribed = 6.0
    telescope.Tsys = 270.
    telescope.OWA = 24.
    telescope.IWA = 3.5
    telescope.qe = 0.9 * 0.75   # Detector QE * charge transfer term
    telescope.Rc = 2.0e-3       # Clock induced charge [counts/pixel/photon]
    telescope.emis = 0.7
    telescope.Dtmax = 1.0e3/3600
    telescope.X = .61

    # Visible channel parameters:
    telescope.resolution = 140.
    telescope.throughput = 0.15
    telescope.darkcurrent = 3e-5
    telescope.readnoise = 0.0
    telescope.lammin = 0.4
    telescope.lammax = 1.030
    
    
    # Running the different coronagraph object to run count rates for AX and AY
    noise_AX = cg.CoronagraphNoise(telescope = telescope,
                            planet = planet,
                            star = star,
                            texp = 10.0,
                            wantsnr = 5.0)
    
    noise_AY = cg.CoronagraphNoise(telescope = telescope,
                            planet = planet,
                            star = star,
                            texp = 10.0,
                            wantsnr = 5.0)
    
    # running count rates to calculate Dt
    noise_AX.run_count_rates(AX_refhr, lamhr, AX_solhr)
    
    noise_AY.run_count_rates(AY_refhr, lamhr, AY_solhr)
    
###################################################################################
# CALCULATING DT
###################################################################################

    # calculating sum of wavelength then using that to divide SN^2 to calculate Dt. (Gialluca et al 2021 equation 9)
    SN = 5.

    sumWav = 0
    i=0
    
    for i in range(len(noise_AX.A)):
        sumWav = sumWav + (( noise_AX.cp[i]**2 * ( noise_AX.A[i] - noise_AY.A[i] )**2 )/( noise_AX.cp[i] + 2*noise_AX.cb[i] ))

    Dts = SN**2/(sumWav)

    Dt = Dts/3600
    print(Dt) 

    print("Done" + str(i) + "\n")
    
    string = str(ax_file) + "  " + str(ay_file) + "  " + "      %g" % (Dt)
    f.write(string + "\n")

f.close()

93.74652871037463
Done133

71.18638516283589
Done133

74.54385421604998
Done133

102.86114220001697
Done133

40.59972092627306
Done133

9.981445141492767
Done133

7.532309812528724
Done133

10.075530778629831
Done133

130.4745293442992
Done133

144.09214820514558
Done133

207.16673794429062
Done133

457.1926258415307
Done133

48.79927994288101
Done133

47.786020494947984
Done133

58.268142342193414
Done133

94.47655029458834
Done133



In [4]:
#################################################################
## Low Water Vapor in Atmosphere at Clear Skies Detection Times##
#################################################################

low_water_list = (['Arthro_clearSky_low_water_mon_toa.rad', 'Arthro_clearSky_low_water_scrubbed_mon_toa.rad',\
                   'Ectoth_um_clearSky_low_water_mon_toa.rad', 'Ectoth_um_clearSky_scrubbed_low_water_mon_toa.rad', \
                   'black_clearSky_low_water_mon_toa.rad', 'black_clearSky_low_water_scrubbed_mon_toa.rad', \
                   'orange_clearsky_low_water_mon_toa.rad', 'orange_scrubbed_clearSky_low_water_mon_toa.rad'])

outfile = "Low_Water_DetectionTimes_10pc.txt"
f = open(outfile, 'w')

for i in np.arange(0, len(low_water_list), 2):
    ax_file = low_water_list[i]
    ay_file = low_water_list[i+1]
    AX_data = newcols(ax_file)
    AY_data = newcols(ay_file)

    lamhr = np.array(AX_data[0])    # Lambda
    AX_solhr = np.array(AX_data[1])    # Solar flux AX 
    AX_radhr = np.array(AX_data[2])    # radiance AX
    AX_refhr = np.array(AX_data[3])    # Reflectance of AX

    AY_solhr = np.array(AY_data[1])    # Solar flux AY
    AY_radhr = np.array(AY_data[2])    # radiance AY
    AY_refhr = np.array(AY_data[3])    # Reflectance of AY
                        
                        
###################################################################################
# RETRIEVING CP0 FOR ALBEDO OF ONE
###################################################################################

    planet = cg.Planet()
    
    # Planet params
    planet.alpha = 90.     # phase angle at quadrature
    planet.Phi   = cg.teleplanstar.lambertPhaseFunction(1)      # phase function at quadrature (already included in SMART run)
    planet.Rp    = 1.0     # Earth radii
    planet.a     = 1.0     # semi-major axis (AU)

    # Stellar params
    star = cg.Star()
    star.Teff  = 5780.   # Sun-like Teff (K)
    star.Rs    = 1.      # star radius in solar radii
    

    # Planetary system params
    planet.distance = 10.    # distance to system (pc)
    planet.Nez  = 1.      # number of exo-zodis

    telescope = cg.Telescope()

    # Set paramaters for luvex
    telescope.diameter = 6.0
    telescope.contrast = 1e-10
    telescope.A_collect = 36.
    telescope.diam_inscribed = 6.0
    telescope.Tsys = 270.
    telescope.OWA = 24.
    telescope.IWA = 3.5
    telescope.qe = 0.9 * 0.75   # Detector QE * charge transfer term
    telescope.Rc = 2.0e-3       # Clock induced charge [counts/pixel/photon]
    telescope.emis = 0.7
    telescope.Dtmax = 1.0e3/3600
    telescope.X = .61

    # Visible channel parameters:
    telescope.resolution = 140.
    telescope.throughput = 0.15
    telescope.darkcurrent = 3e-5
    telescope.readnoise = 0.0
    telescope.lammin = 0.4
    telescope.lammax = 1.030
    
    
    # Running the different coronagraph object to run count rates for AX and AY
    noise_AX = cg.CoronagraphNoise(telescope = telescope,
                            planet = planet,
                            star = star,
                            texp = 10.0,
                            wantsnr = 5.0)
    
    noise_AY = cg.CoronagraphNoise(telescope = telescope,
                            planet = planet,
                            star = star,
                            texp = 10.0,
                            wantsnr = 5.0)
    
    # running count rates to calculate Dt
    noise_AX.run_count_rates(AX_refhr, lamhr, AX_solhr)
    
    noise_AY.run_count_rates(AY_refhr, lamhr, AY_solhr)
    
###################################################################################
# CALCULATING DT
###################################################################################

    # calculating sum of wavelength then using that to divide SN^2 to calculate Dt. (Gialluca et al 2021 equation 9)
    SN = 5.

    sumWav = 0
    i=0
    
    for i in range(len(noise_AX.A)):
        sumWav = sumWav + (( noise_AX.cp[i]**2 * ( noise_AX.A[i] - noise_AY.A[i] )**2 )/( noise_AX.cp[i] + 2*noise_AX.cb[i] ))

    Dts = SN**2/(sumWav)

    Dt = Dts/3600
    print(Dt) 

    print("Done" + str(i) + "\n")
    
    string = str(ax_file) + "  " + str(ay_file) + "  " + "      %g" % (Dt)
    f.write(string + "\n")

f.close()

70.25510802537511
Done133

276.38821791050975
Done133

3850695.986413428
Done133

264.0833531617793
Done133



In [5]:
#################################################################
## Low Water Vapor in Atmosphere at Clear Skies Detection Times##
#################################################################

path=str('/Users/schuylerraeschroerborges/Desktop/Second_Project/')

low_water_list = ([path+'Arthro_clearSky_low_water_mon_toa.rad', path+'Arthro_scrubbed_v3_clearSky_new_lowh2o_toa.rad',\
                   path+'Ectoth_um_clearSky_low_water_mon_toa.rad', path+'Ectoth_um_clearSky_scrubbed_low_water_mon_toa.rad', \
                   path+'black_clearSky_low_water_mon_toa.rad', path+'black_clearSky_low_water_scrubbed_mon_toa.rad', \
                   path+'orange_clearsky_low_water_mon_toa.rad', path+'orange_scrubbed_clearSky_low_water_mon_toa.rad'])

outfile = "Low_Water_DetectionTimes_new_arthro_5pc.txt"
f = open(outfile, 'w')

for i in np.arange(0, len(low_water_list), 2):
    ax_file = low_water_list[i]
    ay_file = low_water_list[i+1]
    AX_data = newcols(ax_file)
    AY_data = newcols(ay_file)

    lamhr = np.array(AX_data[0])    # Lambda
    AX_solhr = np.array(AX_data[1])    # Solar flux AX 
    AX_radhr = np.array(AX_data[2])    # radiance AX
    AX_refhr = np.array(AX_data[3])    # Reflectance of AX

    AY_solhr = np.array(AY_data[1])    # Solar flux AY
    AY_radhr = np.array(AY_data[2])    # radiance AY
    AY_refhr = np.array(AY_data[3])    # Reflectance of AY
                        
                        
###################################################################################
# RETRIEVING CP0 FOR ALBEDO OF ONE
###################################################################################

    planet = cg.Planet()
    
    # Planet params
    planet.alpha = 90.     # phase angle at quadrature
    planet.Phi   = cg.teleplanstar.lambertPhaseFunction(1)      # phase function at quadrature (already included in SMART run)
    planet.Rp    = 1.0     # Earth radii
    planet.a     = 1.0     # semi-major axis (AU)

    # Stellar params
    star = cg.Star()
    star.Teff  = 5780.   # Sun-like Teff (K)
    star.Rs    = 1.      # star radius in solar radii
    

    # Planetary system params
    planet.distance = 5.    # distance to system (pc)
    planet.Nez  = 1.      # number of exo-zodis

    telescope = cg.Telescope()

    # Set paramaters for luvex
    telescope.diameter = 6.0
    telescope.contrast = 1e-10
    telescope.A_collect = 36.
    telescope.diam_inscribed = 6.0
    telescope.Tsys = 270.
    telescope.OWA = 24.
    telescope.IWA = 3.5
    telescope.qe = 0.9 * 0.75   # Detector QE * charge transfer term
    telescope.Rc = 2.0e-3       # Clock induced charge [counts/pixel/photon]
    telescope.emis = 0.7
    telescope.Dtmax = 1.0e3/3600
    telescope.X = .61

    # Visible channel parameters:
    telescope.resolution = 140.
    telescope.throughput = 0.15
    telescope.darkcurrent = 3e-5
    telescope.readnoise = 0.0
    telescope.lammin = 0.4
    telescope.lammax = 1.030
    
    
    # Running the different coronagraph object to run count rates for AX and AY
    noise_AX = cg.CoronagraphNoise(telescope = telescope,
                            planet = planet,
                            star = star,
                            texp = 10.0,
                            wantsnr = 5.0)
    
    noise_AY = cg.CoronagraphNoise(telescope = telescope,
                            planet = planet,
                            star = star,
                            texp = 10.0,
                            wantsnr = 5.0)
    
    # running count rates to calculate Dt
    noise_AX.run_count_rates(AX_refhr, lamhr, AX_solhr)
    
    noise_AY.run_count_rates(AY_refhr, lamhr, AY_solhr)
    
###################################################################################
# CALCULATING DT
###################################################################################

    # calculating sum of wavelength then using that to divide SN^2 to calculate Dt. (Gialluca et al 2021 equation 9)
    SN = 5.

    sumWav = 0
    i=0
    
    for i in range(len(noise_AX.A)):
        sumWav = sumWav + (( noise_AX.cp[i]**2 * ( noise_AX.A[i] - noise_AY.A[i] )**2 )/( noise_AX.cp[i] + 2*noise_AX.cb[i] ))

    Dts = SN**2/(sumWav)

    Dt = Dts/3600
    print(Dt) 

    print("Done" + str(i) + "\n")
    
    string = str(ax_file) + "  " + str(ay_file) + "  " + "      %g" % (Dt)
    f.write(string + "\n")

f.close()

92.86032967299347
Done133

30.67700139351188
Done133

73.406189974112
Done133

38.639864702796075
Done133



In [ ]:
###################################################################################
# Attempts to do the above using dictionaries
###################################################################################

# Converting .rad files to 2D array and assigning columns from mixed files to 2D array
def correctcols(data,isRadfile):
    columnone=[]
    columntwo=[]
    columnfour=[]
    columnthree=[]
    if isRadfile:
        for i in range (0,len(data)):
            columnone.append(float(data[i][0]))
            columnfour.append(float(data[i][3]))
            columnthree.append(float(data[i][2]))
        wave=np.array(columnone)
        rad=np.array(columnfour)
        flux=np.array(columnthree)
    
        reflect=rad/flux
    else:
        for i in range (0,len(data)):
            columnone.append(float(data[i][0]))
            columntwo.append(float(data[i][1]))
            columnthree.append(float(data[i][2]))
            columnfour.append(float(data[i][3]))
        wave=np.array(columnone)
        flux=np.array(columntwo)
        reflect=np.array(columnthree)
        rad=np.array(columnfour)
    
    
    return np.array([wave,flux,rad,reflect])



In [ ]:
""""Loading all data into dictionaries"""

# Read in all files to dictionary (as astropy table)
spectra = dict()
files = os.listdir('.')
files = sorted(files)
for file in files:
    if file.startswith('Arthro'):
        spectra[file] = ascii.read(file, data_start=0,  delimiter='\s')
    elif file.startswith('Ecto'):
        spectra[file] = ascii.read(file, data_start=0,  delimiter='\s')
    elif file.startswith('black'):
        spectra[file] = ascii.read(file, data_start=0,  delimiter='\s')
    elif file.startswith('orangemat'):
        spectra[file] = ascii.read(file, data_start=0,  delimiter='\s')
    else:
        pass

    
# Split into txt and rad files
mixedtxt = dict()
clearrad = dict()
for key in spectra.keys():
    if 'mixed' in key:
        mixedtxt[key] = spectra[key]
    elif '.rad' and 'clear' in key:
        clearrad[key] = spectra[key]
    else:
        pass

for file in mixedtxt:
    mixedtxt[file] = correctcols(mixedtxt[file],False)
    
for file in clearrad:
    clearrad[file] = correctcols(clearrad[file],True)
    
bothmixedclear= dict()

# Merge mixedtxt and clearrad
def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res
     
bothmixedclear = Merge(mixedtxt, clearrad)

# Sort keys in combined dictionary
sortcombined = collections.OrderedDict(sorted(bothmixedclear.items()))

print(sortcombined.keys())

In [ ]:
#########################################
files = sortcombined

outfile = "DetectionTimes.txt"

f = open(outfile, 'w')
#print(files.keys())
#print(files[0])

#print(sortcombined)
###################################################################################
# READING IN DATA FILES FOR AX AND AY
###################################################################################

#print(files.keys())

scrubbed = Table()
notscrubbed = Table()

for key in files.keys():
    if 'scrubbed' in key:
        scrubbed[key] = files[key]
    else:
        notscrubbed[key] = files[key]

for i in range(0, len(scrubbed.columns)):      
    #print(scrubbed.columns[i])
    lamhr_a = scrubbed.columns[i][0]
    #print(np.shape(lamhr_a))
ay_file = []
lamhr = []
AY_solhr = []
AY_radhr = []
AY_refhr = []
for key in scrubbed.keys():  
    ay_file.append(key)
    lamhr.append(scrubbed[key][0])        # Lambda
    AY_solhr.append(scrubbed[key][1])     # Solar flux AY 
    AY_radhr.append(scrubbed[key][2])     # Radiance AY
    AY_refhr.append(scrubbed[key][3])     # Reflectance of AY
    #print(ay_file, AY_refhr)

ax_file = []
AX_solhr = []
AX_radhr = []
AX_refhr = []
for key in notscrubbed.keys():
    ax_file.append(np.array(key))
    AX_solhr.append(notscrubbed[key][1])     # Solar flux AX 
    AX_radhr.append(notscrubbed[key][2])     # Radiance AX
    AX_refhr.append(notscrubbed[key][3])     # Reflectance of AX
#print(ax_file[0])


#for i in ax_file:
    #print(i)
#for i in lamhr:
#    print(i)
#for i in AX_refhr[0]:
    #print(i)
    #for i in AY_refhr[0]:
        #print(i)

#for i in AX_refhr[0] and AY_refhr[0]:
    #print(i)
    #print(np.shape(i))
    #print(np.transpose(i, (1, 98990, 16)).shape)
###################################################################################
# RETRIEVING CP0 FOR ALBEDO OF ONE
###################################################################################
# for i in lamhr and AX_refhr and AY_refhr and AX_solhr and AY_solhr:
#     planet = cg.Planet()
    
#     # Planet params
#     planet.alpha = 90.     # phase angle at quadrature
#     planet.Phi   = cg.teleplanstar.lambertPhaseFunction(1)      # phase function at quadrature (already included in SMART run)
#     planet.Rp    = 1.0     # Earth radii
#     planet.r     = 1.0     # semi-major axis (AU)
    
#     # Stellar params
#     star = cg.Star()
#     star.Teff  = 5780.   # Sun-like Teff (K)
#     star.Rs    = 1.      # star radius in solar radii
    

#     # Planetary system params
#     planet.d    = 10.    # distance to system (pc)
#     planet.Nez  = 1.      # number of exo-zodis

#     telescope = cg.Telescope()

#     # Set paramaters for luvex
#     telescope.diameter = 6.0
#     telescope.contrast = 1e-10
#     telescope.A_collect = 36.
#     telescope.diam_inscribed = 6.0
#     telescope.Tsys = 270.
#     telescope.OWA = 24.
#     telescope.IWA = 3.5
#     telescope.qe = 0.9 * 0.75   # Detector QE * charge transfer term
#     telescope.Rc = 2.0e-3       # Clock induced charge [counts/pixel/photon]
#     telescope.emis = 0.7
#     telescope.Dtmax = 1.0e3/3600
#     telescope.X = .61

#     # Visible channel parameters:
#     telescope.resolution = 140.
#     telescope.throughput = 0.15
#     telescope.darkcurrent = 3e-5
#     telescope.readnoise = 0.0
#     telescope.lammin = 0.515
#     telescope.lammax = 1.030
    
#     # distances = np.arange(5, 15, 0.2)
#     # for i in distances:
#     #    planet.d    = distances   # distance to system (pc)
    
#     # Running the different coronagraph object to run count rates for AX and AY
#     noise_AX = cg.CoronagraphNoise(telescope = telescope,
#                             planet = planet,
#                             star = star,
#                             texp = 10.0,
#                             wantsnr = 5.0)
    
#     noise_AY = cg.CoronagraphNoise(telescope = telescope,
#                             planet = planet,
#                             star = star,
#                             texp = 10.0,
#                             wantsnr = 5.0)
    
#     # running count rates to calculate Dt
#     noise_AX.run_count_rates(AX_refhr[i], lamhr[i], AX_solhr[i])
        
#     noise_AY.run_count_rates(AY_refhr[i], lamhr[i], AY_solhr[i])
    
# ###################################################################################
# # CALCULATING DT
# ###################################################################################

#     # calculating sum of wavelength then using that to divide SN^2 to calculate Dt. (Gialluca et al 2021 equation 9)
#     SN = 5.

#     sumWav = 0
#     i=0
    
#     for i in range(len(noise_AX.A)):
#         sumWav = sumWav + (( noise_AX.cp[i]**2 * ( noise_AX.A[i] - noise_AY.A[i] )**2 )/( noise_AX.cp[i] + 2*noise_AX.cb[i] ))

#         Dts = SN**2/(sumWav)

#         Dt = Dts/3600
#         print(Dt) 

#         print("Done" + str(j) + "\n")
    
#         string = str(ax_file) + "  " + str(ay_file) + "  " + "      %g" % (Dt)
#         f.write(string + "\n")

# f.close()

# f = open(outfile, 'w')
# j = 0
    #AX_data = scrubbed[key]
    #AY_data = notscrubbed[key]
        
    
# Reading in AY
#print(files)
# for key in files.keys():
#     names = key
#     lamhr = files[key][0]
#     solhr = files[key][1]       # Solar flux AX 
#     radhr = files[key][2]       # radiance AX
#     refhr = files[key][3]       # Reflectance of AX
#     print(names, refhr)
#     #print(lamhr)
#     #print(refhr)
    
#for item in files.items():
    #print(item)

# f = open(outfile, 'w')
# j = 0

# for string in files:
#     while j < len(files):

#         #Reading in AX and AY
#         AX_data = files[string][j]
#         j = j+1
    
#         AY_data = files[string][j]
#         j = j+1
    
#         lamhr = np.array(AX_data[0])    # Lambda
#         AX_solhr = np.array(AX_data[1])    # Solar flux AX 
#         AX_radhr = np.array(AX_data[2])    # radiance AX
#         AX_refhr = np.array(AX_data[3])    # Reflectance of AX
    
#         AY_solhr = np.array(AY_data[1])    # Solar flux AY
#         AY_radhr = np.array(AY_data[2])    # radiance AY
#         AY_refhr   = np.array(AY_data[3])    # Reflectance of AY
    
    
# def splitfiles(data,isScrubfile):
#     lam=[]
#     flux=[]
#     rad=[]
#     ref=[]
#     if isScrubfile:
#         for i in data:
#             lam.append(files[string][0])
#             flux.append(files[string][1])       # Solar flux AY 
#             rad.append(files[string][2])       # radiance AY
#             ref.append(files[string][3])       # Reflectance of AY
#     else:
#         for i in data:
#             lam.append(files[string][0])
#             flux.append(files[string][1])       # Solar flux AX 
#             rad.append(files[string][2])       # radiance AX
#             ref.append(files[string][3])       # Reflectance of AX    
    
#     return lam,flux,rad,ref

# AX_data = []
# AY_data = []
# for i in files:
#     AX_data.append(splitfiles(files[i],False))
#     AY_data.append(splitfiles(files[i],True))
# print(np.shape(AX_data))
    

# bio = []
# abio = []
# for i in np.arange(0, len(files), 2):
#     AX_data = bio.append(splitfiles(files[i],False))
#     AY_data = abio.append(splitfiles(files[i],True))
    
# for file in clearrad:
#     clearrad[file] = correctcols(clearrad[file],True)    
# AX_data = splitfiles(files, False)
# print(AX_data)
# AY_data = splitfiles(files, True)
# print(AY_data)
    
    
    
# lamhr = []      # Wavelength
# AY_solhr = []    # Solar flux AY
# AY_radhr = []   # Solar radiance AY
# AY_refhr = []    # Reflectance AY
# AX_solhr = []      # Solar flux AX
# AX_radhr = []     # Radiance AX
# AX_ref = []     # Reflectance AX
# for string in files:
#     if 'scrubbed'in files.keys():
#         #print(string)
#         lamhr.append(files[string][0])
#         AY_solhr.append(files[string][1])    # Solar flux AY
#         AY_radhr.append(files[string][2])     # radiance AY
#         AY_refhr.append(files[string][3])     # Reflectance of AY
#     else: 
#         #print(string)
#         AX_solhr.append(files[string][1])    # Solar flux AX 
#         AX_radhr.append(files[string][2])    # radiance AX
#         AX_rehr = AX_ref.append(files[string][3])    # Reflectance of AX 
#     print('lamhr:',np.shape(lamhr))
#     #print('AX_reflectance:',np.shape(AX_reflectance))
#     print('AX_refhr:',np.shape(AX_refhr))
#     print('AX_solhr:',np.shape(AX_solhr))
#     #print('AX_refhr:',AX_refhr)
#     print('AY_refhr:',np.shape(AY_refhr))
    ###################################################################################
    # RETRIEVING CP0 FOR ALBEDO OF ONE
    ###################################################################################
#     planet = cg.Planet()
    
#     # Planet params
#     planet.alpha = 90.     # phase angle at quadrature
#     planet.Phi   = cg.teleplanstar.lambertPhaseFunction(1)      # phase function at quadrature (already included in SMART run)
#     planet.Rp    = 1.0     # Earth radii
#     planet.r     = 1.0     # semi-major axis (AU)

#     # Stellar params
#     star = cg.Star()
#     star.Teff  = 5780.   # Sun-like Teff (K)
#     star.Rs    = 1.      # star radius in solar radii
    

#     # Planetary system params
#     planet.d    = 10.    # distance to system (pc)
#     planet.Nez  = 1.      # number of exo-zodis

#     telescope = cg.Telescope()

#     # Set paramaters for luvex
#     telescope.diameter = 6.0
#     telescope.contrast = 1e-10
#     telescope.A_collect = 36.
#     telescope.diam_inscribed = 6.0
#     telescope.Tsys = 270.
#     telescope.OWA = 24.
#     telescope.IWA = 3.5
#     telescope.qe = 0.9 * 0.75   # Detector QE * charge transfer term
#     telescope.Rc = 2.0e-3       # Clock induced charge [counts/pixel/photon]
#     telescope.emis = 0.7
#     telescope.Dtmax = 1.0e3/3600
#     telescope.X = .61

#     # Visible channel parameters:
#     telescope.resolution = 140.
#     telescope.throughput = 0.15
#     telescope.darkcurrent = 3e-5
#     telescope.readnoise = 0.0
#     telescope.lammin = 0.515
#     telescope.lammax = 1.030
    
#     # distances = np.arange(5, 15, 0.2)
#     # for i in distances:
#     #    planet.d    = distances   # distance to system (pc)
    
#     # Running the different coronagraph object to run count rates for AX and AY
#     noise_AX = cg.CoronagraphNoise(telescope = telescope,
#                             planet = planet,
#                             star = star,
#                             texp = 10.0,
#                             wantsnr = 5.0)
    
#     noise_AY = cg.CoronagraphNoise(telescope = telescope,
#                             planet = planet,
#                             star = star,
#                             texp = 10.0,
#                             wantsnr = 5.0)
    
#     # running count rates to calculate Dt
#     noise_AX.run_count_rates(AX_refhr, lamhr, AX_solhr)
    
#     noise_AY.run_count_rates(AY_refhr, lamhr, AY_solhr)
    
#     ###################################################################################
#     # CALCULATING DT
#     ###################################################################################

#     # calculating sum of wavelength then using that to divide SN^2 to calculate Dt. (Gialluca et al 2021 equation 9)
#     SN = 5.

#     sumWav = 0
#     i=0
    
#     for i in range(len(noise_AX.A)):
#         sumWav = sumWav + (( noise_AX.cp[i]**2 * ( noise_AX.A[i] - noise_AY.A[i] )**2 )/( noise_AX.cp[i] + 2*noise_AX.cb[i] ))

#     Dts = SN**2/(sumWav)

#     Dt = Dts/3600
#     print(Dt) 

#     print("Done" + str(j) + "\n")
    
#     string = ax_file + "  " + ay_file + "  " + "      %g" % (Dt)
#     f.write(string + "\n")

# f.close()

# f = open(outfile, 'w')
# j = 0

# while j < len(files):

#     #Reading in AX and AY
#     AX_data = files[j]
#     j = j+1
    
#     AY_data = files[j]
#     j = j+1
    
#     lamhr = np.array(AX_data[0])    # Lambda
#     AX_solhr = np.array(AX_data[1])    # Solar flux AX 
#     AX_radhr = np.array(AX_data[3])    # radiance AX
#     AX_refhr = np.array(AX_data[2])    # Reflectance of AX

#     AY_solhr = np.array(AY_data[1])    # Solar flux AY
#     AY_radhr = np.array(AY_data[3])    # radiance AY
#     AY_refhr   = np.array(AY_data[2])    # Reflectance of AY


In [ ]:
###################################################################################
# FORMER NOISE MODEL
###################################################################################

from astropy.io import ascii 
from astropy.table import Table
from astropy.table import Column
import numpy as np 
import coronagraph as cg
import matplotlib.pyplot as plt
import math
import glob


fileNames = glob.glob('*_clearSky_toa.rad')
dtArray = []
outfile = "DetectionTimes.txt"

###################################################################################
# READING IN DATA FILES FOR AX AND AY
###################################################################################

#Reading in AY
AY_data = ascii.read("black_clearSky_mon_toa.rad", data_start=0, delimiter='\s')
    
f = open(outfile, 'w')

for file in fileNames:
    print(file)
    
    if(file != "black_clearSky_mon_toa.rad" and file != "black_clearSky_mon_toa.rad" and file != "orange_clearSky_mon_toa.rad"):
        
        print("in")
        #Reading in AX
        AX_data = ascii.read(file, data_start=0, delimiter='\s')

        lamhr = np.array(AX_data['col1'][:])    # Lambda
        AX_solhr = np.array(AX_data['col3'][:])    # Solar flux AX 
        AX_radhr = np.array(AX_data['col4'][:])    # radiance AX

        AX_refhr   = (AX_radhr/AX_solhr)    # Reflectance of AX

        AY_solhr = np.array(AY_data['col3'][:])    # Solar flux AY
        AY_radhr = np.array(AY_data['col4'][:])    # radiance AY

        AY_refhr   = (AY_radhr/AY_solhr)    # Reflectance of AY

        ###################################################################################
        # RETRIEVING CP0 FOR ALBEDO OF ONE
        ###################################################################################

        # Planet params
        alpha = 90.     # phase angle at quadrature
        Phi   = cg.teleplanstar.lambertPhaseFunction(alpha)      # phase function at quadrature (already included in SMART run)
        Rp    = 1.0     # Earth radii
        r     = 1.0     # semi-major axis (AU)

        # Stellar params
        Teff  = 5780.   # Sun-like Teff (K)
        Rs    = 1.      # star radius in solar radii

        # Planetary system params
        d    = 10.     # distance to system (pc)
        Nez  = 1.      # number of exo-zodis

        
        lam, dlam, Ax, q, Cratio, cpx, csp, cz, cez, cD, cR, cth, cc, DtSNRx = \
                cg.count_rates(AX_refhr, lamhr, AX_solhr, alpha, Phi, Rp, Teff, Rs, r, d, Nez,\
                               lammax=1.6)

        lam, dlam, Ay, q, Cratio, cp, csp, cz, cez, cD, cR, cth, cc, DtSNR = \
                cg.count_rates(AY_refhr, lamhr, AY_solhr, alpha, Phi, Rp, Teff, Rs, r, d, Nez,\
                               lammax=1.6)

        ###################################################################################
        # CALCULATING DT
        ###################################################################################
    
        # check if the sum of all the noise is correct or if it should be computed like in ty's paper
        cb = cz + cez + csp + cD + cR + cth + cc

        # maybe need to add the S/N^2 for every refletance in the array but might do that already
        SN = 5.

        sumWav = 0
        for i in range(len(Ax)):
            sumWav = sumWav + (( cpx[i]**2 * ( Ax[i] - Ay[i] )**2 )/( cpx[i] + 2*cb[i] ))


        Dts = SN**2/(sumWav)

        Dt = Dts/3600
        print(Dt) 
        

        string = file + "      %g" % (Dt)
        f.write(string + "\n")

f.close()



#def luvex(channel):

#    telescope = cg.Telescope()

    # Set paramaters for luvex
#        telescope.diameter = 6.0
#        telescope.contrast = 1e-10
#        telescope.A_collect = 36.
#         telescope.diam_inscribed = 6.0
#         telescope.Tsys = 270.
#         telescope.OWA = 24.
#         telescope.IWA = 3.5
#         telescope.qe = 0.9 * 0.75   # Detector QE * charge transfer term
#         telescope.Rc = 2.0e-3       # Clock induced charge [counts/pixel/photon]
#         telescope.emis = 0.7
#         telescope.Dtmax = 1.0e3/3600
#         telescope.X = .61
        
#         if channel.lower() == "vis".lower():
#             # Visible channel parameters:
#             telescope.resolution = 140.
#             telescope.throughput = 0.15
#             telescope.darkcurrent = 3e-5
#             telescope.readnoise = 0.0
#             telescope.lammin = 0.515
#             telescope.lammax = 1.030
            
#        elif channel.lower() == "UV".lower():
#            # UV channel parameters:
#            telescope.resolution = 7.
#            telescope.throughput = 0.085
#            telescope.darkcurrent = 3e-5
#            telescope.readnoise = 0.0
#            telescope.lammin = 0.200
#            telescope.lammax = 0.525
            
#        elif channel.lower() == "NIR".lower():
            # NIR channel parameters:
#            telescope.resolution = 70.
#            telescope.throughput = 0.22
#            telescope.darkcurrent = 2e-3
#            telescope.readnoise = 2.0
#            telescope.lammin = 1.00
#            telescope.lammax = 2.00
            
#         else:
#             print("Unknown `channel`")
#             return None
#     return telescope

In [ ]:
luvex = cg.Telescope.default_luvoir
noise = cg.CoronagraphNoise(telescope = luvex,
                            planet = planet,
                            star = star,
                            texp = 10.0,
                            wantsnr = 5.0)
noise.run_count_rates(AX_refhr, lamhr, AX_solhr)
print(noise.SNRt)